In [1]:
import numpy as np
import sys, os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import measure
import cv2
from skimage import morphology
from skimage.morphology import disk, square, dilation, erosion
from skimage import data
from skimage.feature import Cascade
from EyeCenterLocator import EyeCenterLocator
from FaceAligner import FaceAligner
import imutils
from scipy import signal
from skimage import filters
from skimage import feature
import skimage.io as io
from scipy import fftpack
from scipy.signal import convolve2d
from skimage.util import random_noise
from skimage.exposure import rescale_intensity
from PIL import Image
import requests

In [2]:
def skinDetection(sourceImage):
    min_YCrCb = np.array([0,133,77],np.uint8)
    max_YCrCb = np.array([255,173,127],np.uint8)

    # Convert image to YCrCb
    imageYCrCb = cv2.cvtColor(sourceImage,cv2.COLOR_RGB2YCR_CB)

    # Find region with skin tone in YCrCb image
    skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)

    # Do contour detection on skin region
    contours, hierarchy = cv2.findContours(skinRegion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contour on the source image
    max_contour_area = -1
    max_contour_index = -1
    for i, c in enumerate(contours):
        area = cv2.contourArea(c)
        if area > max_contour_area:
            max_contour_index = i
            max_contour_area = area
    sourceImage = np.zeros(sourceImage.shape).astype(sourceImage.dtype)
    if max_contour_index > -1: 
        cv2.drawContours(sourceImage, contours, max_contour_index, (255, 255, 255), -1)
    sourceImage = cv2.cvtColor(sourceImage,cv2.COLOR_RGB2GRAY)
    return sourceImage

In [3]:
def GetMouthMap(image_RGB):
    # transform the image to the YCbCr space
    image_YCrCb = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2YCR_CB)
    
    # extracting YCbCr space components
    Y = image_YCrCb[:,:,0]
    Cr = np.asarray(image_YCrCb[:,:,1], np.int16)
    Cb = np.asarray(image_YCrCb[:,:,2], np.int16)
    
    skin = skinDetection(image_RGB)
    
    # calculating the MouthMap
    n = np.count_nonzero(skin==255)
        
    eta = 0
    
    try:
        v_1 = np.power(Cr[skin==255],2)
        v_1 =  v_1/(np.amax(v_1)/255.0)

        v_2 = np.divide(Cr[skin==255],Cb[skin==255])
        v_2 =  v_2/(np.amax(v_2)/255.0)

        eta = 0.95 * np.sum(v_1)/np.sum(v_2)
    except ValueError:  #raised if `v_1` is empty or `v_2`
        pass
    
    val_1 = np.power(Cr,2)
    val_1 = val_1/(np.amax(val_1)/255.0)

    val_2 = Cr/Cb
    val_2 = val_2/(np.amax(val_2)/255.0)

    MouthMap = val_1 * np.power(val_1 - eta * val_2, 2)
    MouthMap = MouthMap/(np.amax(MouthMap)/255.0)
    
    mask = disk(10)
    MouthMap = dilation(MouthMap, selem=mask)
    
    MouthMap = cv2.bitwise_and(MouthMap,  MouthMap,mask =skin)
    
    MouthMap = np.asarray(MouthMap, dtype='uint8')
    
    ret2,MouthMap = cv2.threshold(MouthMap,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return MouthMap

In [4]:
def MouthMapROI(image_RGB):
    mouth_ROI = face[2*face.shape[0]//3:face.shape[0],face.shape[1]//3:2*face.shape[1]//3]
    start_x = w//3
    start_y = 2*h//3
    mouth = GetMouthMap(mouth_ROI)
    contours, hierarchy = cv2.findContours(mouth, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    l = len(contours)
    #for contour in contours:
    x_c = 0
    y_c = 0
    w_c = 0
    h_c = 0
    if len(contours) > 0:
        x_c,y_c,w_c,h_c = cv2.boundingRect(contours[-1])
        x_c += (start_x)
        y_c += (start_y)
    return x_c,y_c,w_c,h_c 

In [5]:
haar_file = 'haarcascade_frontalface_default.xml'
webcam = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(haar_file)
while True:
    (_, im) = webcam.read()
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 4)
    l = 0
    for (x, y, w, h) in faces:
        face = cv2.cvtColor(im[y :y + h , x :x + w ], cv2.COLOR_BGR2RGB)
        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
        x_c,y_c,w_c,h_c  = MouthMapROI(face)
        x_c += (x)
        y_c += (y)
        
        im = cv2.circle(im, (x_c, y_c + h_c//2), 2, (255,0,255), -1)
        im = cv2.circle(im, (x_c + w_c, y_c + h_c//2), 2, (255,0,255), -1)
        
        #im = cv2.rectangle(im, (x_c, y_c), (x_c + w_c, y_c + h_c), (255, 255, 0), 2)
        break
    cv2.imshow('output',im)
    key = cv2.waitKey(10)
    if key == 27:
        break
    
cv2.destroyAllWindows()
webcam.release()

In [9]:
cv2.destroyAllWindows()
webcam.release()